# A brief visualization of the the daily exchange rates per Euro 1999-2021

In [ ]:
import logging

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
%matplotlib inline

In [ ]:
# reading the data
exchange_rates = pd.read_csv(r'data/euro-daily-hist_1999_2021.csv')

## Visualising some of the first rows

In [ ]:
exchange_rates.head()

## Visualising some of the last rows

In [ ]:
exchange_rates.tail()

## Checking for a brief data information
Here we can see that the data is not fully complete, there are some NaN's along the 45 columns and 5877 rows of the dataset. One other thing that is noticeble is that only some columns are assuming numeric values ```(float64)```, almost every single column is stored as a string ```(object)```.

In [ ]:
exchange_rates.info()

## Data cleaning
The data will be cleaning on the following aspects:
* Renaming the column names: for the exchange rates, only the removal of unecessary characters, for the ```Period\Unit:```, it will be called ```Time   ```;
* Changing the date column to ```datetime```;
* Sorting the ```Time``` values in an ascending order;
* Reseting the index (due to the sorting) and droping it; 

In [ ]:
# renaming the column names
exchange_rates.columns = [column.replace('[', '').replace(' ]', '') for column in exchange_rates.columns]
exchange_rates.columns = [column.replace(' ', '_') for column in exchange_rates.columns]
exchange_rates.rename(columns={r'Period\Unit:': 'Time'}, inplace=True)

# changing the type of the Time column to datetime
exchange_rates['Time'] = pd.to_datetime(exchange_rates['Time'])

# sorting the dates
exchange_rates.sort_values('Time', ascending=True, inplace=True)

# reseting the index and droping it
exchange_rates.reset_index(drop=True, inplace=True)

In [ ]:
exchange_rates.head()

## Delimiting our problem
For the next analysis and visualizations, our focus will be on the Brazilian REAL and the North American DOLLAR to EURO exchange rates.

In [ ]:
# segmenting the dataset for the dollar
euro2dollar = exchange_rates[['Time', 'US_dollar']].copy()
# segmenting the dataset for the real
euro2real = exchange_rates[['Time', 'Brazilian_real']].copy()

## Checking for data incosistency
As we check for any strange or unreadable values over the segmented data, we encounter that 62 ```'-'``` values are in the dollar's data, and 61 on the real's data. Because of this, only the data without these values will be filtered to be used.

In [ ]:
euro2dollar.US_dollar.value_counts()

In [ ]:
euro2real.Brazilian_real.value_counts()

In [ ]:
# filtering for the non '-' values on both dataframes
euro2dollar = euro2dollar[euro2dollar['US_dollar'] != '-']
euro2real = euro2real[euro2real['Brazilian_real'] != '-']

## Converting the ```string``` to ```numeric``` ones
For both datasets, its necessary to adequate the data to a numeric type, so, the columns ```euro2dollar['US_dollar']``` and ```euro2real['Brazilian_real']``` will be converted to ``float64````.

In [ ]:
# converting the column types of both dataframes
euro2dollar['US_dollar'] = euro2dollar['US_dollar'].astype(float)
euro2real['Brazilian_real'] = euro2real['Brazilian_real'].astype(float)

## Verifying the evolutiong of the exchangin rates (on respect with the Euro)

In [ ]:
# the temporary plotting style
style.use('seaborn-talk')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

ax1.plot(euro2dollar['Time'], euro2dollar['US_dollar'])
ax2.plot(euro2real['Time'], euro2real['Brazilian_real'])

for ax in [ax1, ax2]:
    ax.set_xlabel('Year')
    ax.set_ylabel('Coin Value')
    
    for location in ['right', 'top']:
        ax.spines[location].set_visible(False)

ax1.set_title('Evolution of the Euro-Dollar Exchange Rate', size=20, weight='bold')
ax2.set_title('Evolution of the Euro-Real Exchange Rate', size=20, weight='bold')

plt.show()

## Analysing the rolling means
Visualizaing for different rolling window sizes, how the data is plotted to us. For long-term trends, the curves are smoother as we can see bellow.

In [ ]:
fig = plt.figure(figsize=(12, 15))

rolling_window = [1, 7, 30, 50, 100, 365]
plot_number = range(1, 7)
for i, rolling_mean in zip(plot_number, rolling_window):
    ax = fig.add_subplot(3, 2, i)
    
    for location in ['right', 'top']:
        ax.spines[location].set_visible(False)
    
    plt.title('Rolling Window:' + str(rolling_mean), weight='bold')
    ax.plot(euro2dollar['Time'], euro2dollar['US_dollar'].rolling(rolling_mean).mean(), color='lightblue', label='dollar', alpha=0.8)
    ax.plot(euro2real['Time'], euro2real['Brazilian_real'].rolling(rolling_mean).mean(), color='red', label='Real')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# setting the rollng mean for euro2real and euro2dollar
euro2dollar['rolling_mean'] = euro2dollar['US_dollar'].rolling(30).mean()
euro2real['rolling_mean'] = euro2real['Brazilian_real'].rolling(30).mean()

## Visualizing the Euro-Dollar exchange rate for last 3 United States presidents
Here, our objective is to see how the euro to dollar parity fluctuated over the passage of the last 3 presidents of United States (without including Binden). For this, we need to know that here:

* George W. Bush: 2001 ~ 2009;
* Barack Obama: 2009 ~ 2017;
* Donald Trump: 2017 ~ 2021;

So we have to filter our data, since it started on 1999.

In [ ]:
# segmenting the full dataset to its respective president   
df_all_presidents = euro2dollar.copy().loc[(euro2dollar['Time'].dt.year >= 2001) & (euro2dollar['Time'].dt.year < 2021)]
bush = df_all_presidents.copy().loc[df_all_presidents['Time'].dt.year < 2009]
obama = df_all_presidents.copy().loc[(euro2dollar['Time'].dt.year >= 2009) & (euro2dollar['Time'].dt.year < 2017)]
trump = df_all_presidents.copy().loc[(df_all_presidents['Time'].dt.year >= 2017) & (df_all_presidents['Time'].dt.year < 2021)]

In [ ]:
style.use('fivethirtyeight')

# creating the subplots
plt.figure(figsize=(20, 10))

# two rows: one with 3 graphs side-to-side and the second with only a bigger graph
ax1 = plt.subplot(2, 3, 1)
ax2 = plt.subplot(2, 3, 2)
ax3 = plt.subplot(2, 3, 3)
ax4 = plt.subplot(2, 1, 2)

# changes to all the subplots
axes = [ax1, ax2, ax3, ax4]
for ax in axes:
    ax.set_ylim(0.8, 1.7)
    ax.set_yticks([1.0, 1.2, 1.4, 1.6])
    ax.set_yticklabels(['1.0', '1.2','1.4', '1.6'],  alpha=0.3)
    ax.grid(alpha=0.5)    

# ploting for Bush
ax1.plot(bush['Time'], bush['rolling_mean'], color='#00FFFF')
ax1.set_xticklabels(['', '2001', '', '2003', '', '2005', '', '2007', '', '2009'], alpha=0.3)
ax1.text(12500, 1.85, 'BUSH', fontsize=18, weight='bold', color='#00FFFF')
ax1.text(12330, 1.8, '(2001-2009)', weight='bold', alpha=0.3)

# plotting for Obama
ax2.plot(obama['Time'], obama['rolling_mean'], color='#00BFFF')
ax2.set_xticklabels(['', '2009', '', '2011', '', '2013', '', '2015', '', '2017'], alpha=0.3)
ax2.text(15300.0, 1.85, 'OBAMA', fontsize=18, weight='bold', color='#00BFFF')
ax2.text(15200.0, 1.8, '(2009-2017)', weight='bold', alpha=0.3)

# plotting for Trump
ax3.plot(trump['Time'], trump['rolling_mean'], color='#4B7FFF')
ax3.set_xticklabels(['2017', '', '2018', '', '2019', '', '2020', '', '2021'], alpha=0.3)
ax3.text(17700.0, 1.85, 'TRUMP', fontsize=18, weight='bold', color='#4B7FFF')
ax3.text(17640.0, 1.8, '(2017-2021)', weight='bold', alpha=0.3)

# plotting for all presidents together
ax4.plot(bush['Time'], bush['rolling_mean'], color='#00FFFF')
ax4.plot(obama['Time'], obama['rolling_mean'], color='#00BFFF')
ax4.plot(trump['Time'], trump['rolling_mean'], color='#4B7FFF')

ax4.plot(bush['Time'], bush['rolling_mean'], color='#00FFFF')
ax4.plot(obama['Time'], obama['rolling_mean'], color='#00BFFF')
ax4.plot(trump['Time'], trump['rolling_mean'], color='#4B7FFF')
ax4.grid(alpha=0.5)
ax4.set_xticks([])

# title and subtile being added
ax1.text(13000.0, 2.12, 'EURO-USD rate averaged 1.22 under the last three US presidents', fontsize=22, weight='bold')
ax1.text(13000.0, 2.0, '''EURO-USD exchange rates under George W. Bush (2001 - 2009), Barack Obama (2009-2017),
and Donald Trump (2017-2021)''', fontsize=16)

# adding the signature
ax4.text(10500.0, 0.65, '©Ariel Alsina' + ' '*230 + 'Source: European Central Bank', 
        color = '#f0f0f0', backgroundcolor = '#4d4d4d',
        size=14)


plt.show()